In [2]:
import pandas as pd
import requests
from google.oauth2 import service_account


In [3]:
credentials = service_account.Credentials.from_service_account_file("connection-123-892e002c2def.json")

In [6]:

def fetch_eth_price() -> pd.DataFrame:
    url = 'https://api.coingecko.com/api/v3/coins/tether/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': '365',
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    # Extract prices, market caps, and total volumes
    market_caps = data['market_caps']
    total_volumes = data['total_volumes']

    # Create DataFrames for each metric
    df_market_caps = pd.DataFrame(market_caps, columns=['timestamp', 'market_cap'])
    df_volumes = pd.DataFrame(total_volumes, columns=['timestamp', 'total_volume'])

    # Merge all data on timestamp
    df = df_market_caps.merge(df_volumes, on='timestamp')

    # Parse timestamps in UTC, normalize to midnight, then shift back one day
    ts_utc = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    dates_utc = ts_utc.dt.normalize() - pd.Timedelta(days=1)

    # Use the shifted date as the label for the "closing" day
    df['date'] = dates_utc.dt.date
    df = df.drop(columns=['timestamp']).sort_values('date').reset_index(drop=True)

    # Ensure one row per date (in case of any duplicates)
    df = df.groupby('date', as_index=False).last()
    df = df.rename(columns={'date': 'timestamp'})

    return df



In [7]:
data = fetch_eth_price()

In [8]:
data.head()

,timestamp,market_cap,total_volume
0,2024-08-28,1.181357e+11,4.605388e+10
1,2024-08-29,1.181729e+11,4.701164e+10
2,2024-08-30,1.182220e+11,4.881389e+10
3,2024-08-31,1.182304e+11,2.076913e+10
4,2024-09-01,1.182957e+11,2.948821e+10
